In [161]:
# Limites para investimento (inferior e superior)
limite = 1000000
limiteinf = int(limite*0.8)

#Custo de cada opção de investimento
custo = [470000,400000,170000,270000,340000,230000,50000,440000,100000,100000]

#Rendimento de cada opção de investimento  
retorno = [410000,330000,140000,250000,320000,320000,90000,190000,500000,150000]

#Retorno Relativo
retornoRel = [r/c for c, r in zip(custo, retorno)]

#Riscos - 1 Baixo - 2 Medio - 3 Alto
risco=[1,1,1,2,2,3,3,1,3,3]


In [162]:
from pulp import *

#Classe com o solver 
class Investimento:
    def __init__(self, custo, retorno, limite, limiteinf, risco):
        self.custo = custo
        self.retorno = retorno
        self.limite = limite
        self.limiteinf = limiteinf
        self.opcoes = len(custo)
        self.risco = risco
        
        # Criando um problema
        self.problema = LpProblem("Otimização de investimento", LpMaximize)
        
        #Criando lista de variavel de decisão binaria
        self.list_var = [LpVariable(f"Investmento{i+1}", 0, 1, LpBinary) for i in range(self.opcoes)]
        
        #Funcao Objetivo
        self.problema += lpSum([retorno[i] * self.list_var[i] for i in range(self.opcoes)])
  
        #RESTRIÇÕES
        self.problema += lpSum([custo[i] * self.list_var[i] for i in range(self.opcoes)]) <= limite 
        self.problema += lpSum([custo[i] * self.list_var[i] for i in range(self.opcoes)]) >= limiteinf
        #Relação entre as opções
        self.problema += self.list_var[0] + self.list_var[4] <= 1
        self.problema += self.list_var[1] - self.list_var[3] <= 0
        # Riscos
        self.problema += lpSum(self.list_var[i] for i in range(self.opcoes) if value(risco[i]) == 3) <= 1
        self.problema += lpSum(self.list_var[i] for i in range(self.opcoes) if value(risco[i]) == 2) <= 2
        self.problema += lpSum(self.list_var[i] for i in range(self.opcoes) if value(risco[i]) == 1) >= 1
        #diversificação
        self.problema += lpSum(self.list_var[i] for i in range(self.opcoes)) == 4
        
    def solver(self):
        self.problema.solve()
        
        result = {"Investimento": [], "Custo": [], "Retorno Esperado": [], "Opção": [], "Risco": []}
        for i in range(self.opcoes):
            result["Investimento"].append(i+1)
            result["Custo"].append(self.custo[i])
            result["Retorno Esperado"].append(int(self.retorno[i]*self.custo[i])) #Normalizado para retorno normal 
            result["Opção"].append(int(value(self.list_var[i])))
            result["Risco"].append(self.risco[i])
        return result
    
#instancia da classe 
invest = Investimento(custo, retornoRel, limite, limiteinf, risco)
result = invest.solver()

c:\Users\luiiz\AppData\Local\Programs\Python\Python39\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [163]:
print("Investimento\tCusto\t\tRetorno Esp.\tRisco")
print("___________________________________________________________")
soma_custo=0
soma_rendimento=0
for i in range(len(result['Opção'])):
  if result['Opção'][i] == 1:
    print(f"{result['Investimento'][i]}\t\t{result['Custo'][i]}\t\t{result['Retorno Esperado'][i]}\t\t{result['Risco'][i]}")
    soma_custo += result['Custo'][i]
    soma_rendimento += result['Retorno Esperado'][i] 
print("___________________________________________________________")
print(f"{'Total:'}\t\t{soma_custo}\t\t{int(soma_rendimento)}")

Investimento	Custo		Retorno Esp.	Risco
___________________________________________________________
3		170000		140000		1
4		270000		250000		2
5		340000		320000		2
9		100000		500000		3
___________________________________________________________
Total:		880000		1210000


In [164]:
import unittest

class Teste(unittest.TestCase):
    
    def test_custo(self):
        invest = Investimento(custo, retornoRel, limite, limiteinf, risco)
        self.result = invest.solver()
        
        # Testa se o custo total dos investimentos não ultrapassa o limite de investimento
        custoTotal = sum(self.result['Custo'][i] * self.result["Opção"][i] for i in range(len(self.result["Opção"])) if result['Opção'][i] == 1)
        self.assertLessEqual(custoTotal, limite)
        self.assertGreaterEqual(custoTotal, limiteinf)
        
     
    def teste_restricoes(self):
        invest = Investimento(custo, retornoRel, limite, limiteinf, risco)
        self.result = invest.solver()
        
        # Testa se as restrições das opcoes foram aplicadas corretamente
        self.assertLessEqual(self.result["Opção"][0] + self.result["Opção"][4], 1)
        self.assertLessEqual(self.result["Opção"][1] - self.result["Opção"][3], 0)
       
        # Testa se as restrições de riscos foram aplicadas corretamente
        risco_tres = sum(self.result["Opção"][i] for i in range(len(self.result["Opção"])) if self.result['Risco'][i] == 3)
        self.assertLessEqual(risco_tres, 1)
        
        risco_dois = sum(self.result["Opção"][i] for i in range(len(self.result["Opção"])) if self.result['Risco'][i] == 2)
        self.assertLessEqual(risco_dois, 2)
        
        risco_um = sum(self.result["Opção"][i] for i in range(len(self.result["Opção"])) if self.result['Risco'][i] == 1)
        self.assertEqual(risco_um, 1)
        
        # Testa se as restrições de diversificacao de foi aplicadas corretamente
        self.assertEqual(sum(self.result["Opção"][i] for i in range(len(self.result["Opção"]))), 4)
 
    def teste_retorno(self):
        invest = Investimento(custo, retornoRel, limite, limiteinf, risco)
        self.result = invest.solver()
        
        # Testa se o retorno esperado é igual ao retorno esperado do resultado
        expected_return = lpSum([self.result['Retorno Esperado'][i] * self.result["Opção"][i] for i in range(len(self.result["Opção"]))])
        self.assertEqual(expected_return, self.result["Retorno Esperado"])
        
unittest.main(argv=[''], exit=False)

c:\Users\luiiz\AppData\Local\Programs\Python\Python39\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
...
----------------------------------------------------------------------
Ran 3 tests in 0.182s

OK


In [8]:
li = [10, 20, 10, '10', 20,30,40,40,40]
st = set(li)
print(st)

{40, 10, 20, '10', 30}
